### Создание разметки для силы комбинаций

    Порядок действий:
    
        1. Создать строковое представление для 52 карт
        2. Создать словарь старшенства карт вида {строки из 1 пункта: range(0, 13)}
        3. Создание функций для определения ранга комбинации (от старшей карты до флешрояль) и старшинства карт в комбинации
        4. Создание всех возможных 5-картовых комбинаций из строк с помощью itetools.combinations
        5. Разметка силы комбинаций внутри каждого ранга комбинаций
        6. Генерация рандомного числа с зафиксированным сидом для 52 карт
        7. Создание и сохранение словаря вида {карта: сгенерированное число}
        8. Создание всех возможных 5-картовых комбинаций из чисел с помощью itetools.combinations
        9. Сложение чисел по столбцам (2-D -> 1-D)
        10. Создание словаря вида {сумма_чисел_в_комбинации: ранг_комбинации)

In [1]:
import itertools
import collections
import pickle
import re

import pandas as pd
import numpy as np

In [2]:
rang = tuple(['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A'])
suits = tuple(['c', 'h', 'd', 's'])

cards = tuple([''.join([x[0], x[1]]) for x in itertools.product(rang, suits)])

In [3]:
d = {key: value for (key, value) in zip(rang, range(len(rang)))}

In [4]:
def create_street(dict_card):
    koleso = set([list(d.keys())[-1]] + list(d.keys())[:4])
    l = [koleso]
    for i in range(len(list(dict_card.keys()))-4):
        l.append(set(list(dict_card.keys())[i: i +5]))
    return l

def rang_combination(combination, dict_card, street):
    
    card = list(re.search(r'\d{1}|[^chds]', card, re.IGNORECASE).group() for card in combination)
    suit = list(re.search(r'[chds]', card, re.IGNORECASE).group() for card in combination)    
    
    card_cnt = collections.Counter(card)
    
    card = set(card)
    suit = set(suit)
    
    t = {dict_card[key]: value for (key, value) in card_cnt.items()}
    dc = [x[0] for x in sorted(t.items(), key = lambda kv: [kv[1], kv[0]], reverse = True)]
    
    if len(card) == 5 and len(suit) != 1 and card not in street:
        return (0, dc)#highcard
    elif len(card) == 4:
        return (1, dc) #pair
    elif len(card) == 3 and set(card_cnt.values()) == {2, 2, 1}:
        return (2, dc) #twopairs
    elif len(card) == 3:
        return (3, dc) #set
    elif len(card) == 5 and len(suit) != 1 and card in street:
        if dc == [12, 3, 2, 1, 0]:
            return (4, [3, 2, 1, 0, -1]) # koleso
        return (4, dc) #street
    elif len(card) == 5 and len(suit) == 1 and card not in street:
        return (5, dc) # flash
    elif len(card) == 2 and set(card_cnt.values()) == {3, 2}:
        return (6, dc) #fullhouse
    elif len(card) == 2:
        return (7, dc) #care
    elif len(card) == 5 and len(suit) == 1 and card in street[:-1]:
        if dc == [12, 3, 2, 1, 0]:
            return (8, [3, 2, 1, 0, -1]) #koleso
        return (8, dc) #street-flash
    else:
        return (9, dc) #flash-royal

In [5]:
all_comb = tuple(itertools.combinations(cards, 5))

In [6]:
h = {x: rang_combination(x, d, create_street(d)) for x in all_comb}

In [7]:
a = pd.DataFrame({'hand': h.keys(), 'power': h.values()})

In [8]:
a['cards'] = [x[1] for x in a['power']]
a['power'] = [x[0] for x in a['power']]

a[['card1', 'card2', 'card3', 'card4', 'card5']] = pd.DataFrame(a.cards.to_list(), index=a.index).fillna(0).astype(np.int8)

a = a.drop(columns='cards')

a = a.sort_values(by=['power', 'card1', 'card2', 'card3', 'card4', 'card5'], ascending=False).reset_index(drop=True)

a['tmp'] = pd.Series((a['power'] != a['power'].shift(1)) | \
  (a['card1'] != a['card1'].shift(1)) | \
  (a['card2'] != a['card2'].shift(1)) | \
  (a['card3'] != a['card3'].shift(1)) | \
  (a['card1'] != a['card1'].shift(1)) | \
  (a['card5'] != a['card5'].shift(1)))

a['mark'] = a.tmp.cumsum()-1
a = a.drop(columns='tmp')

In [11]:
np.random.seed(100)
m = np.unique(np.random.randint(0, 15000, 250))**3
num_cards = np.random.choice(m, 52, False)

In [12]:
card_dict = {key: value for (key, value) in zip(cards, num_cards)}

In [13]:
filehandler = open("card_dict.dat", 'wb') 
pickle.dump(card_dict , filehandler, pickle.HIGHEST_PROTOCOL)
filehandler.close()

In [14]:
all_num = np.array(tuple(itertools.combinations(num_cards, 5)))

all_num_sum = np.sum(all_num, axis= 1)

h1 = {x[0]: x[1] for x in zip(all_comb, all_num_sum)}

b = pd.DataFrame({'hand': h1.keys(), 'num': h1.values()})

In [15]:
df = a.loc[:, ['hand', 'mark']].merge(b, how='inner', on='hand', validate='1:1')

In [16]:
d = {key:value for (key, value) in zip(df['num'], df['mark'])}

In [17]:
np_key = np.array([x for x in df["num"]], dtype=np.int64)

In [20]:
np_value = np.array([x for x in df["mark"]], dtype=np.int16)

In [23]:
np.save("np_key.npy", np_key)
np.save("np_value.npy", np_value)

In [16]:
df.loc[:, ["num", "mark"]].to_csv("comb_data.csv", index=False)

In [14]:
filehandler = open("comb_dict.dat", 'wb') 
pickle.dump(d , filehandler, pickle.HIGHEST_PROTOCOL)
filehandler.close()